In [46]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from datetime import datetime
import numpy as np
import seaborn as sns
import scipy as sc
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

In [47]:
warnings.filterwarnings("ignore")
df = pd.read_csv(r"books.csv",sep=",", error_bad_lines=False, index_col="bookID")
df.columns = df.columns.str.replace(" ", "")
df = df[df.ratings_count != 0]
df = df[df['num_pages'].apply(lambda x: isinstance(x, (int, float)))]

b'Skipping line 3350: expected 12 fields, saw 13\nSkipping line 4704: expected 12 fields, saw 13\nSkipping line 5879: expected 12 fields, saw 13\nSkipping line 8981: expected 12 fields, saw 13\n'


In [48]:
df["is_eng"] = df["language_code"].str.contains('eng|en-').astype(int)
df.drop(columns=["language_code"]);

In [49]:
df["publication_date"] = pd.to_datetime(df["publication_date"], errors='coerce')
df = df.dropna()
df.head()

,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,is_eng
bookID,,,,,,,,,,,,
1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,2006-09-16,Scholastic Inc.,1
2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,2004-09-01,Scholastic Inc.,1
4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,2003-11-01,Scholastic,1
5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,2004-05-01,Scholastic Inc.,1
8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,2004-09-13,Scholastic,1


In [50]:
if df['num_pages'].dtype == 'float64':
    print("all good with num_pages")
else:
    print("some values in num_pages are not floats")

some values in num_pages are not floats


In [51]:
specific_date_str = "2020-01-01"
specific_date = pd.to_datetime(specific_date_str)

df["time_since_published"] = df["publication_date"].apply(lambda x: (specific_date - x).days)
df["log_time_since_published"] = np.log(df["time_since_published"])

In [52]:
df.describe()

,average_rating,isbn13,num_pages,ratings_count,text_reviews_count,is_eng,time_since_published,log_time_since_published
count,11041.000000,1.104100e+04,11041.000000,1.104100e+04,11041.000000,11041.000000,11041.000000,11040.000000
mean,3.943213,9.759719e+12,336.703741,1.807515e+04,546.046735,0.948555,7058.049543,8.795880
std,0.295221,4.446139e+11,240.978566,1.129056e+05,2585.749958,0.220913,3001.854683,0.352289
min,1.000000,8.987060e+09,0.000000,1.000000e+00,0.000000,0.000000,-90.000000,4.919981
25%,3.780000,9.780345e+12,194.000000,1.100000e+02,9.000000,1.000000,5205.000000,8.557375
50%,3.960000,9.780573e+12,300.000000,7.660000e+02,48.000000,1.000000,6150.000000,8.724207
75%,4.140000,9.780872e+12,416.000000,5.061000e+03,241.000000,1.000000,7823.000000,8.964823
max,5.000000,9.790008e+12,6576.000000,4.597666e+06,94265.000000,1.000000,43829.000000,10.688051


In [53]:
df.head()
print(df.columns)
def is_audio(row):
    if row['num_pages'] < 30 or "audio" in row["publisher"]:
        return 1
    else:
        return 0

Index(['title', 'authors', 'average_rating', 'isbn', 'isbn13', 'language_code',
       'num_pages', 'ratings_count', 'text_reviews_count', 'publication_date',
       'publisher', 'is_eng', 'time_since_published',
       'log_time_since_published'],
      dtype='object')


In [54]:
df["r_count_per_yr"] = df.ratings_count / df.time_since_published
df["t_count_per_yr"] = df.text_reviews_count / df.time_since_published
df["audio"] = df.apply(lambda row: is_audio(row), axis=1)

In [55]:
df.head()

,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,is_eng,time_since_published,log_time_since_published,r_count_per_yr,t_count_per_yr,audio
bookID,,,,,,,,,,,,,,,,,
1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,2006-09-16,Scholastic Inc.,1,4855,8.487764,431.656025,5.683007,0
2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,2004-09-01,Scholastic Inc.,1,5600,8.630522,384.494107,5.218036,0
4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,2003-11-01,Scholastic,1,5905,8.683555,1.072481,0.041321,0
5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,2004-05-01,Scholastic Inc.,1,5723,8.652248,408.803949,6.347196,0
8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,2004-09-13,Scholastic,1,5588,8.628377,7.413744,0.029349,0


In [56]:
df = df[df.audio == 0]

In [57]:
features = ['ratings_count','text_reviews_count','r_count_per_yr','t_count_per_yr','num_pages','is_eng','time_since_published']
X = df.loc[:, features]
y = df.loc[:, ['average_rating']]

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .80)

In [59]:
reg = DecisionTreeRegressor(max_depth = 2, random_state = 0)

In [60]:
reg.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=2, random_state=0)

In [33]:
X_test.head(1)

,ratings_count,text_reviews_count,r_count_per_yr,t_count_per_yr,num_pages,is_eng
bookID,,,,,,
42485,83,16,0.009037,0.001742,128,1


In [61]:
score = reg.score(X_test, y_test)
print(score)

0.04033382453948664
